In [1]:
import requests
import json
from pprint import pprint 
import pandas as pd
from sqlalchemy import create_engine, inspect
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
import sqlite3
Base = declarative_base()

In [2]:
# Pull in combined data for airlines

American_Alaskan = pd.read_csv("Resources/CSV/Combined/AA_concat.csv")
Delta = pd.read_csv("Resources/CSV/Combined/Delta_Airlines.csv")
Southwest = pd.read_csv("Resources/CSV/Combined/Southwest_Airlines.csv")
United = pd.read_csv("Resources/CSV/Combined/United_Airlines.csv")

# Clean up column headers so that all data is matching
American_Alaskan.rename(columns={'Origin Airport':'Origin_Airport',
                          'Carrier Code':'Carrier_Code',
                          'Destination Airport':'Destination_Airport',
                          'Date (MM/DD/YYYY)':'Date'},
                 inplace=True)

AA = American_Alaskan[American_Alaskan.columns.drop('Unnamed: 0')]

Delta.rename(columns={'Carrier Code':'Carrier_Code',
                      'Date (MM/DD/YYYY)':'Date'},
                 inplace=True)
Delta['Carrier_Code'] = "Delta Airlines"

Southwest.rename(columns={'Date (MM/DD/YYYY)':'Date'},
                inplace=True)

United.rename(columns={'Date (MM/DD/YYYY)':'Date'},
                inplace=True)

In [3]:
# Create SQLite engine
engine = create_engine('sqlite:///flights.sqlite', echo=False)
inspector = inspect(engine)
Base.metadata.create_all(engine)
session= Session(engine)

In [4]:
# Create table and append airline flights data
con=engine
con.execute('DROP TABLE IF EXISTS flights')
AA.to_sql('flights', con)
Delta.to_sql('flights', con, if_exists='append', index=False)
Southwest.to_sql('flights', con, if_exists='append', index=False)
United.to_sql('flights', con, if_exists='append', index=False)

# Count the data stored within the newly created database
engine.execute("SELECT COUNT(*) FROM flights").fetchall()

[(2365604,)]

In [5]:
# Database too large. Drop data from all years except 2018.

engine.execute("DELETE FROM flights WHERE Date LIKE '%2019%'")
engine.execute("DELETE FROM flights WHERE Date LIKE '%2017%'")
engine.execute("DELETE FROM flights WHERE Date LIKE '%2016%'")
engine.execute("DELETE FROM flights WHERE Date LIKE '%2015%'")
engine.execute("DELETE FROM flights WHERE Date LIKE '%2014%'")

engine.execute("SELECT COUNT(*) FROM flights").fetchall()

[(496336,)]

In [6]:
# Use vacuum to rebuild the main data base and change the file size from 217MB to 46,232KB
engine.execute("vacuum")

In [7]:
# Put Flights sqlite database into a data frame and rename column headers that were dropped

flights_db = pd.DataFrame(engine.execute("SELECT * FROM flights").fetchall())

flights_db = flights_db.rename(columns={1:'Carrier_Code', 2:'Date', 
                  5:'Destination_Airport', 8:'Scheduled_Elapsed_Time', 
                  9:'Actual_Elapsed_Time', 10:'Departure_Delay',
                  13:'Delay_Carrier', 14:'Delay_Weather', 
                  15:'Delay_National_Aviation_Sys', 16:'Delay_Security',
                  17:'Delay_Late_Aircraft_Arrival', 18:'Origin_Airport'
                          })

flights_db = flights_db.drop(columns={0, 3, 4, 6, 7, 11, 12})

flights_db

,Carrier_Code,Date,Destination_Airport,Scheduled_Elapsed_Time,Actual_Elapsed_Time,Departure_Delay,Delay_Carrier,Delay_Weather,Delay_National_Aviation_Sys,Delay_Security,Delay_Late_Aircraft_Arrival,Origin_Airport
0,American Airlines,01/01/2018,DFW,155.0,153.0,1.0,0.0,0.0,0.0,0.0,0.0,ATL
1,American Airlines,01/01/2018,DFW,156.0,132.0,-2.0,0.0,0.0,0.0,0.0,0.0,ATL
2,American Airlines,01/01/2018,DFW,153.0,135.0,-4.0,0.0,0.0,0.0,0.0,0.0,ATL
3,American Airlines,01/01/2018,DFW,151.0,161.0,-3.0,0.0,0.0,0.0,0.0,0.0,ATL
4,American Airlines,01/01/2018,LAX,317.0,296.0,-4.0,0.0,0.0,0.0,0.0,0.0,ATL
...,...,...,...,...,...,...,...,...,...,...,...,...
496331,United Airlines,12/31/2018,DEN,140.0,164.0,-13.0,0.0,0.0,0.0,0.0,0.0,PDX
496332,United Airlines,12/31/2018,ORD,228.0,243.0,-4.0,0.0,0.0,0.0,0.0,0.0,PDX
496333,United Airlines,12/31/2018,SFO,108.0,87.0,62.0,2.0,0.0,0.0,0.0,39.0,PDX
496334,United Airlines,12/31/2018,DEN,139.0,158.0,3.0,0.0,0.0,22.0,0.0,0.0,PDX


In [8]:
flights_db = flights_db.drop(columns={"Departure_Delay","Delay_Carrier", "Delay_Weather", 
                                      "Delay_National_Aviation_Sys", "Delay_Security", "Delay_Late_Aircraft_Arrival"})
flights_db

,Carrier_Code,Date,Destination_Airport,Scheduled_Elapsed_Time,Actual_Elapsed_Time,Origin_Airport
0,American Airlines,01/01/2018,DFW,155.0,153.0,ATL
1,American Airlines,01/01/2018,DFW,156.0,132.0,ATL
2,American Airlines,01/01/2018,DFW,153.0,135.0,ATL
3,American Airlines,01/01/2018,DFW,151.0,161.0,ATL
4,American Airlines,01/01/2018,LAX,317.0,296.0,ATL
...,...,...,...,...,...,...
496331,United Airlines,12/31/2018,DEN,140.0,164.0,PDX
496332,United Airlines,12/31/2018,ORD,228.0,243.0,PDX
496333,United Airlines,12/31/2018,SFO,108.0,87.0,PDX
496334,United Airlines,12/31/2018,DEN,139.0,158.0,PDX


In [9]:
# jsonify the dataframe

flights_db.to_json(r"Resources/json/flights.json", orient='table')

In [10]:
airlines = flights_db.groupby("Carrier_Code")
airlines.head()

,Carrier_Code,Date,Destination_Airport,Scheduled_Elapsed_Time,Actual_Elapsed_Time,Origin_Airport
0,American Airlines,01/01/2018,DFW,155.0,153.0,ATL
1,American Airlines,01/01/2018,DFW,156.0,132.0,ATL
2,American Airlines,01/01/2018,DFW,153.0,135.0,ATL
3,American Airlines,01/01/2018,DFW,151.0,161.0,ATL
4,American Airlines,01/01/2018,LAX,317.0,296.0,ATL
115941,Alaska Airlines,01/01/2018,SEA,345.0,331.0,ATL
115942,Alaska Airlines,01/01/2018,SEA,345.0,343.0,ATL
115943,Alaska Airlines,01/02/2018,SEA,345.0,311.0,ATL
115944,Alaska Airlines,01/02/2018,SEA,345.0,328.0,ATL
115945,Alaska Airlines,01/03/2018,SEA,345.0,340.0,ATL
